# [모듈 3.1] NCF 훈련 파이프라인

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 **약 5분** 소요 됩니다.

- 0. SageMaker Model Building Pipeline 개요
- 1. 파이프라인 변수 및 환경 설정
- 2. 파이프라인 스텝 단계 정의

    - (1) 모델 승인 상태 변경 람다 스텝    
    - (2) 배포할 세이지 메이커 모델 스텝 생성
    - (3) 모델 앤드 포인트 배포를 위한 람다 스텝 생성    
    
- 3. 모델 빌딩 파이프라인 정의 및 실행
- 4. Pipleline 캐싱 및 파라미터 이용한 실행
- 5. 정리 작업
    
---

# 1. 파이프라인 변수 및 환경 설정



In [2]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('./src')

In [3]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker.Session().default_bucket()

print("bucket: ", bucket)
print("role: ", role)

sm_client = boto3.client('sagemaker', region_name=region)

%store -r model_package_group_name

bucket:  sagemaker-us-east-1-057716757052
role:  arn:aws:iam::057716757052:role/mlops-blog-ncf-gsmoon


In [4]:
%store -r inference_docker_image
# %store -r artifact_path
%store -r s3_input_data_uri
%store -r byom_artifact
%store -r bucket
%store -r prefix
%store -r artifact_path

In [5]:
print("s3_input_data_uri: \n", s3_input_data_uri)
print("bucket: \n", bucket)
print("prefix: \n", prefix)

s3_input_data_uri: 
 s3://sagemaker-us-east-1-057716757052/NCFModel/data
bucket: 
 sagemaker-us-east-1-057716757052
prefix: 
 NCFModel


In [6]:
project_prefix = prefix

---
# 2. 패키징 코드
- 패키징할 폴더 생성
- 추론 코드의 폴더 지정 후에 패키징 폴더에 복사
- 패키징 폴더를 source.tar.gz 압축 후에 S3에 업로딩
- 



## 2.1. src 폴더를 source.tar.gz 로 압축

In [7]:
import os

package_dir = 'code_pkg'
os.makedirs(package_dir, exist_ok=True)


In [8]:
code_dir = '../src'

In [9]:
%%sh -s {package_dir} {code_dir}

package_dir=$1
code_dir=$2

cd $package_dir # 폴더 생성
echo $PWD
rm -rf ./*
cp -r $code_dir/*.py  .  # src py 모두 카피
cp -r $code_dir/*.txt  .  # src 파일 모두 카피
tar -czvf source.tar.gz * # model.tar.gz 파일 생성

/home/ec2-user/SageMaker/Neural-Collaborative-Filtering-On-SageMaker/3_MLOps/1_sm_pipeline/1_training_pipeline/code_pkg
back-iam_repackage_model_artifact.py
config.py
data_utils.py
evaluate.py
evaluation.py
iam_helper.py
iam_repackage_model_artifact.py
inference.py
inference_utils.py
lambda_helper.py
model.py
pipeline_util.py
repackage.py
requirements.txt
train_lib.py
train_metric.py
train_pipeline.py
train.py


## 2.2. S3 에 업로드

In [10]:
import sagemaker
sagemaker_session  = sagemaker.session.Session()
prefix='ncf/code'
bucket = sagemaker.session.Session().default_bucket()
print("bucket: ", bucket)

source_path = os.path.join(package_dir, 'source.tar.gz')
source_artifact = sagemaker_session.upload_data(source_path, bucket, prefix)
print("source_artifact: \n", source_artifact)

bucket:  sagemaker-us-east-1-057716757052
source_artifact: 
 s3://sagemaker-us-east-1-057716757052/ncf/code/source.tar.gz


### 모델 아피텍트 위치 확인

In [11]:
print("artifact_path: \n", artifact_path)

artifact_path: 
 s3://sagemaker-us-east-1-057716757052/pytorch-training-2022-10-14-13-19-13-618/output/model.tar.gz


# 3. 세이지 메이커 파이프라인 생성

## 3.1. 모델 빌딩 파이프라인 변수 및 세션 생성

파이프라인에 인자로 넘길 변수는 아래 크게 3가지 종류가 있습니다.
- 모델 레지스트리에 모델 등록시에 모델 승인 상태 값    


In [12]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)

# 입력 데이터
s3_data_loc = ParameterString(
    name="InputData",
    default_value=s3_input_data_uri,
)


model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)


### 3.1.1 로컬 모드 설정 
- 로컬 모드 사용을 위해서는 Estimator, Pipeline() 오브젝트 생성시에 인자로서 sagemaker_session 에 LocalPipelineSession() 를 할당해야 합니다.
- 모델 훈련 스텝은 로컬 모드가 가능합니다. 
    - 람다 스텝, 모델 등록 스텝은 지원을 하지 않음.
- Tip : 노트북 하단에서 Pipeline() 를 선언시에 steps 부분을 주석 처리하면서 사용하세요.
``` python
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        s3_data_loc,                
        model_approval_status,        
    ],
    sagemaker_session=pipeline_session,
    steps=[step_train],    # 로컬 모드 사용시
#   steps=[step_train, step_repackage_lambda, step_model_registration],
```

In [13]:
# LOCAL_MODE = True # 로컬 모드시 사용
LOCAL_MODE = False # 클라우드 모드시 사용
if LOCAL_MODE:
    from sagemaker.workflow.pipeline_context import LocalPipelineSession
    pipeline_session = LocalPipelineSession()
    print("### --> Local Mode")
else:
    from sagemaker.workflow.pipeline_context import PipelineSession
    pipeline_session = PipelineSession()
    print("### --> Cloud Mode")    

### --> Cloud Mode


## 3.2. 파이프라인 스텝 단계 정의

### 3.2.1 모델 훈련 스텝

####  하이퍼파라미터 세팅

In [14]:
host_hyperparameters = {'epochs': 1, 
                       'lr': 0.001,
                       'batch_size': 256,
                       'top_k' : 10,
                       'dropout' : 0.0,
                       'factor_num' : 32,
                       'num_layers' : 3,
                       'num_ng' : 4,
                       'test_num_ng' : 99,                   
                    }  

#### 훈련 메트릭을 CloudWatch 에서 보기
- 개발자 가이드
    - [Monitor and Analyze Training Jobs Using Amazon CloudWatch ](https://docs.amazonaws.cn/en_us/sagemaker/latest/dg/training-metrics.html#define-train-metrics)

In [15]:
metric_definitions=[
       {'Name': 'HR', 'Regex': 'HR=(.*?);'},
       {'Name': 'NDCG', 'Regex': 'NDCG=(.*?);'},
       {'Name': 'Loss', 'Regex': 'Loss=(.*?);'}        
    ]


####  Estimator 생성

Estimator 생성시에 인자가 필요 합니다. 주요한 인자만 보겠습니다.


In [16]:
from sagemaker.pytorch import PyTorch

estimator_output_path = f's3://{bucket}/{project_prefix}/training_jobs'
print("estimator_output_path: \n", estimator_output_path)


instance_type = 'ml.p3.2xlarge'
instance_count = 1

host_estimator = PyTorch(
    entry_point="train.py",    
    source_dir='src',    
    role=role,
    output_path = estimator_output_path,    
    framework_version='1.8.1',
    py_version='py3',
    disable_profiler = True,
    instance_count=instance_count,
    instance_type=instance_type,
    session = pipeline_session,
    hyperparameters=host_hyperparameters,
    metric_definitions = metric_definitions
    
)
# host_estimator.fit(s3_inputs, 
#                    experiment_config = experiment_config, # 실험 설정 제공                   
#                    wait=False)

estimator_output_path: 
 s3://sagemaker-us-east-1-057716757052/NCFModel/training_jobs


#### 모델 훈련 스탭 생성


In [17]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name= "NCF-Training",
    estimator=host_estimator,
    inputs={
        "train": TrainingInput(
            s3_data= s3_data_loc
        ),
        "test": TrainingInput(
            s3_data= s3_data_loc
        ),        
    }
)

### 3.2.2 모델 아티펙트 리패키징 람다 스텝 정의

####  람다 Role 생성

In [18]:
from src.iam_helper import create_lambda_role

lambda_role = create_lambda_role("lambda-deployment-role")
print("lambda_role: \n", lambda_role)

Using ARN from existing role: lambda-deployment-role
lambda_role: 
 arn:aws:iam::057716757052:role/lambda-deployment-role


#### 람다 스텝 정의

In [19]:
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)

# import time 
# current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())
# function_name = "sagemaker-lambda-step-approve-model-deployment-" + current_time
function_name = "sagemaker-lambda-step-repackage-model-artifact"

print("function_name: \n", function_name)
# Lambda helper class can be used to create the Lambda function
func_repackage_model = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="src/iam_repackage_model_artifact.py",
    handler="iam_repackage_model_artifact.lambda_handler",
)

output_param_1 = LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)
output_param_3 = LambdaOutput(output_name="S3_Model_URI", output_type=LambdaOutputTypeEnum.String)

step_repackage_lambda = LambdaStep(
    name="LambdaRepackageStep",
    lambda_func=func_repackage_model,
    inputs={
        "source_path" : source_artifact,
        "model_path": step_train.properties.ModelArtifacts.S3ModelArtifacts,
#        "model_path": artifact_path,        
        "bucket" : bucket,
        "prefix" : "ncf/repackage/model"
    },
    outputs=[output_param_1, output_param_2, output_param_3],
)


function_name: 
 sagemaker-lambda-step-repackage-model-artifact


### 3.2.3 모델 등록 스텝

#### 모델 그룹 생성

- 참고
    - 모델 그룹 릭스팅 API:  [ListModelPackageGroups](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_ListModelPackageGroups.html)
    - 모델 지표 등록: [Model Quality Metrics](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/model-monitor-model-quality-metrics.html)

In [20]:
model_package_group_name = f"NCFModel"
model_package_group_input_dict = {
 "ModelPackageGroupName" : model_package_group_name,
 "ModelPackageGroupDescription" : "Sample model package group"
}
response = sm_client.list_model_package_groups(NameContains=model_package_group_name)
if len(response['ModelPackageGroupSummaryList']) == 0:
    print("No model group exists")
    print("Create model group")    
    
    create_model_pacakge_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
    print('ModelPackageGroup Arn : {}'.format(create_model_pacakge_group_response['ModelPackageGroupArn']))    
else:
    print(f"{model_package_group_name} exitss")

NCFModel exitss


#### 모델 등록 스텝 정의

In [21]:
%%time 

from sagemaker.workflow.model_step import ModelStep


from sagemaker.model import Model

image_uri = '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.8.1-gpu-py3'
# model_data = 's3://sagemaker-us-east-1-057716757052/repackage_model_artifact-2022-10-23-05-43-28-940/output/repackage/model.tar.gz'

model = Model(
    image_uri=image_uri,
    model_data = step_repackage_lambda.properties.Outputs["S3_Model_URI"],
    role=role,
    sagemaker_session=pipeline_session,
)


register_model_step_args = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.g4dn.xlarge", "ml.p2.xlarge"],
    transform_instances=["ml.g4dn.xlarge"],
    model_package_group_name=model_package_group_name,
    # approval_status=model_approval_status,
)

step_model_registration = ModelStep(
   name="RegisterModel",
   step_args=register_model_step_args,
)

CPU times: user 2.48 ms, sys: 416 µs, total: 2.89 ms
Wall time: 2.4 ms


/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sagemaker/workflow/pipeline_context.py:233: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


# 4.모델 빌딩 파이프라인 정의 및 실행
위에서 정의한 아래의 4개의 스텝으로 파이프라인 정의를 합니다.
-     steps=[step_process, step_train, step_create_model, step_deploy],
- 아래는 약 1분 정도 소요 됩니다.  이후 아래와 같이 실행 결과를 스튜디오에서 확인할 수 있습니다.


- ![deployment-pipeline.png](img/deployment-pipeline.png)

In [24]:
from sagemaker.workflow.pipeline import Pipeline

project_prefix = 'ncf-pipeline-nb'

pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        s3_data_loc,                
        model_approval_status,        
    ],
    sagemaker_session=pipeline_session,
#    steps=[step_train],    
    steps=[step_train, step_repackage_lambda, step_model_registration],

)



### 파이프라인을 SageMaker에 제출하고 실행하기 


In [25]:
import json

definition = json.loads(pipeline.definition())
# print(" definition : \n", definition)
pipeline.upsert(role_arn=role)
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

실행이 완료될 때까지 기다립니다.

In [26]:
execution.wait()

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

In [27]:
execution.list_steps()

[{'StepName': 'RegisterModel-RegisterModel',
  'StartTime': datetime.datetime(2022, 11, 13, 5, 38, 58, 170000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 11, 13, 5, 38, 59, 288000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:057716757052:model-package/ncfmodel/33'}}},
 {'StepName': 'LambdaRepackageStep',
  'StartTime': datetime.datetime(2022, 11, 13, 5, 38, 45, 110000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 11, 13, 5, 38, 57, 248000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'Lambda': {'Arn': 'arn:aws:lambda:us-east-1:057716757052:function:sagemaker-lambda-step-repackage-model-artifact',
    'OutputParameters': [{'Name': 'body', 'Value': '"Repackaging is Done"'},
     {'Name': 'S3_Model_URI',
      'Value': 's3://sagemaker-us-east-1-057716757052/ncf/repackage/model/model.tar.gz'},
     {'Name': 'statusCode', 'Value': '200.

# 5. 정리 작업


In [ ]:
# step_approve_lambda.properties.Outputs["other_key"]
# execution.list_steps()[1]['Metadata']['Lambda']['OutputParameters'][0]['Value']